In [13]:
!uv pip install hiclass==4.9
!uv pip install scikit-learn==1.2.2

Resolved 7 packages in 89ms                                          
Uninstalled 2 packages in 49ms
Installed 2 packages in 17ms                                
 - numpy==2.0.0
 + numpy==1.26.4
 - scipy==1.17.0
 + scipy==1.12.0
Resolved 5 packages in 4ms                                           
Uninstalled 1 package in 20ms
Installed 1 package in 24ms                                 
 - scikit-learn==1.8.0
 + scikit-learn==1.2.2


In [1]:
#!uv pip uninstall hiclass scikit-learn

In [35]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from hiclass.MultiLabelLocalClassifierPerNode import MultiLabelLocalClassifierPerNode

In [3]:

# # Define data
# X_train = [[1, 2], [3, 4], [5, 6]]
# X_test = [[1, 2], [3, 4], [5, 6]]

# # Define labels
# Y_train = np.array(
#     [
#         [["Mammal", "Human"], ["Fish"]],  # Mermaid
#         [["Mammal", "Human"], ["Mammal", "Bovine"]],  # Minotaur
#         [["Mammal", "Human"]],  # just a Human
#     ],
#     dtype=object,
# )


In [4]:
import torch, json, os, logging
import numpy as np
import pickle
from torch.utils.data import DataLoader
from hmc.datasets.manager.dataset_manager import initialize_dataset_experiments
from torch.utils.data import Dataset
import networkx as nx
import pandas as pd
import logging
import ast

2026-01-26 23:09:20.692642: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-26 23:09:21.738972: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [5]:
from hmc.utils.path.dir import __load_json__

In [6]:
from sklearn.impute import SimpleImputer
from sklearn import preprocessing

In [7]:
dataset_path = '/home/bruno/git/hmc-torch/data'

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/bruno/git/hmc-torch/notebooks/hiclass/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [9]:
dataset_kwargs = {
    "device": "cuda",
    "dataset_type": "arff",
    "is_global": False,
    "dataset_path": dataset_path,
}

In [10]:
dataset = initialize_dataset_experiments('seq_FUN', **dataset_kwargs)

2026-01-26 23:09:23,016 - INFO - Loading dataset from /home/bruno/git/hmc-torch/data/HMC_data_arff/datasets_FUN/seq_FUN/seq_FUN.train.arff
2026-01-26 23:09:24,128 - INFO - Shape of edges matrix: {1: (18, 80), 2: (80, 178), 3: (178, 142), 4: (142, 77), 5: (77, 4)}
2026-01-26 23:09:24,129 - INFO - Parsed ARFF file: /home/bruno/git/hmc-torch/data/HMC_data_arff/datasets_FUN/seq_FUN/seq_FUN.train.arff
2026-01-26 23:09:24,129 - INFO - Number of matrix: 5
2026-01-26 23:09:24,137 - INFO - Loading dataset from /home/bruno/git/hmc-torch/data/HMC_data_arff/datasets_FUN/seq_FUN/seq_FUN.valid.arff
2026-01-26 23:09:24,703 - INFO - Shape of edges matrix: {1: (18, 80), 2: (80, 178), 3: (178, 142), 4: (142, 77), 5: (77, 4)}
2026-01-26 23:09:24,704 - INFO - Parsed ARFF file: /home/bruno/git/hmc-torch/data/HMC_data_arff/datasets_FUN/seq_FUN/seq_FUN.valid.arff
2026-01-26 23:09:24,704 - INFO - Number of matrix: 5
2026-01-26 23:09:24,709 - INFO - Loading dataset from /home/bruno/git/hmc-torch/data/HMC_data_

In [17]:
X_train = dataset.train.X

In [18]:
X_test = dataset.test.X

In [19]:
Y_test = dataset.test.Y_nodes

In [20]:
Y_train = dataset.train.Y_nodes

In [21]:
def convert_label(y_train):
    new_label = []
    for y in y_train:
        labels = []
        for label in y.split('.'):
            labels.append(label)
        new_label.append(labels)
    return new_label

In [22]:
new_y = np.array([convert_label(y_train) for y_train in Y_train],dtype=object)

In [ ]:
# Use decision tree classifiers for every node
tree = DecisionTreeClassifier()
classifier = MultiLabelLocalClassifierPerNode(local_classifier=tree)


In [39]:
# Use decision tree classifiers for every node
clas = RandomForestClassifier()
clf = svm.SVC(probability=True)
classifier = MultiLabelLocalClassifierPerNode(local_classifier=clf)


In [40]:
# Train local classifier per node
classifier.fit(X_train, new_y)

2026-01-26 23:35:16,931 - INFO - Creating digraph from 1701 3D labels
2026-01-26 23:35:17,135 - INFO - Detected 18 roots
2026-01-26 23:35:17,136 - INFO - Initializing local classifiers
2026-01-26 23:35:17,142 - INFO - Initializing siblings binary policy
2026-01-26 23:35:17,142 - INFO - Fitting local classifiers
2026-01-26 23:35:45,730 - INFO - Cleaning up variables that can take a lot of disk space


MultiLabelLocalClassifierPerNode(local_classifier=SVC(probability=True))

In [42]:
# Predict
predictions = classifier.predict(X_test)

2026-01-26 23:36:03,330 - INFO - Predicting
2026-01-26 23:36:03,332 - INFO - Predicting for node '02'
2026-01-26 23:36:03,489 - INFO - Predicting for node '20'
2026-01-26 23:36:03,811 - INFO - Predicting for node '10'
2026-01-26 23:36:04,095 - INFO - Predicting for node '11'
2026-01-26 23:36:04,392 - INFO - Predicting for node '16'
2026-01-26 23:36:04,695 - INFO - Predicting for node '38'
2026-01-26 23:36:04,744 - INFO - Predicting for node '99'
2026-01-26 23:36:04,760 - INFO - Predicting for node '34'
2026-01-26 23:36:04,909 - INFO - Predicting for node '14'
2026-01-26 23:36:05,230 - INFO - Predicting for node '01'
2026-01-26 23:36:05,682 - INFO - Predicting for node '12'
2026-01-26 23:36:05,827 - INFO - Predicting for node '32'
2026-01-26 23:36:05,999 - INFO - Predicting for node '42'
2026-01-26 23:36:06,239 - INFO - Predicting for node '43'
2026-01-26 23:36:06,384 - INFO - Predicting for node '18'
2026-01-26 23:36:06,450 - INFO - Predicting for node '40'
2026-01-26 23:36:06,529 - IN

In [43]:
predictions[3]

array([['01', '05', '25', '', '']], dtype='<U122')

In [44]:
Y_test[3]

['02.11', '02.13.03']